In [35]:
import pandas as pd
from scipy.io import arff

# Define interaction values for event booking system
interaction_values = {
    b'browse': 2,
    b'view': 3,
    b'purchase': 7,
    b'add_to_wishlist': 5,
    b'remove_from_wishlist': 0,
    b'view_friend_profile': 3
}

# Relevant interactions indicating high interest
relevant_interactions = {b'purchase', b'add_to_wishlist'}

# Load the datasets
data, meta = arff.loadarff('cleaned_user_behaviour_new.arff')
user_behavior_df = pd.DataFrame(data)
events_df = pd.read_csv('events.csv')
user_preferences_df = pd.read_csv('user_preferred_categories.csv')
users_df = pd.read_csv('users.csv')
friends_df = pd.read_csv('friends.csv')

# Map interaction values and fill NaN values
user_behavior_df['interaction_value'] = user_behavior_df['interaction_type'].map(interaction_values)

# Generate user relevant items
def get_user_relevant_items(user_behavior_df, relevant_interactions):
    relevant_df = user_behavior_df[user_behavior_df['interaction_type'].isin(relevant_interactions)]
    user_relevant_items = relevant_df.groupby('user_id')['event_id'].apply(set).to_dict()
    return user_relevant_items

# Generate user_relevant_items dynamically
user_relevant_items = get_user_relevant_items(user_behavior_df, relevant_interactions)

# Precision and Recall Calculation
def calculate_precision_recall(recommended_products, relevant_products):
    recommended_set = set(recommended_products)
    true_positives = recommended_set.intersection(relevant_products)
    precision = len(true_positives) / len(recommended_set) if recommended_set else 0
    recall = len(true_positives) / len(relevant_products) if relevant_products else 0
    return {"precision": precision, "recall": recall}

# Recommendation Approach 1: Popular Events
def recommend_popular_events(n=5, user_id=None):
    popular_events = (user_behavior_df.groupby('event_id')['interaction_value']
                      .sum()
                      .reset_index()
                      .sort_values(by='interaction_value', ascending=False)
                      .head(n))
    recommendations = pd.merge(popular_events, events_df[['event_id', 'event_name', 'location', 'ticket_price']], 
                               on='event_id', how='left').dropna()
    
    # Calculate precision and recall if user_id is provided
    if user_id in user_relevant_items:
        relevant_items = user_relevant_items[user_id]
        precision_recall = calculate_precision_recall(recommendations['event_id'], relevant_items)
        print(f"Popular Events - Precision: {precision_recall['precision']:.2f}, Recall: {precision_recall['recall']:.2f}")
    return recommendations[['event_id', 'event_name', 'location', 'ticket_price']]

# Approach 2: Category-Based Recommendation
def recommend_by_selected_category(user_id, n=10):
    user_categories = user_preferences_df[user_preferences_df['user_id'] == user_id]['category_id'].tolist()
    filtered_events = events_df[events_df['category_id'].isin(user_categories)]
    recommendations = filtered_events.sample(n=min(n, len(filtered_events)))
    
    # Calculate precision and recall
    if user_id in user_relevant_items:
        relevant_items = user_relevant_items[user_id]
        precision_recall = calculate_precision_recall(recommendations['event_id'], relevant_items)
        print(f"Category-Based - Precision: {precision_recall['precision']:.2f}, Recall: {precision_recall['recall']:.2f}")
    return recommendations[['event_id', 'event_name', 'location', 'ticket_price', 'category_id']]

# Approach 3: Profile-Based Filtering with Weighted Features
def profile_based_recommend(user_id, n=10):
    user_profile = user_behavior_df[user_behavior_df['user_id'] == user_id].copy()
    user_profile['weight'] = user_profile['interaction_value']
    user_interests = user_profile.groupby('event_id')['weight'].sum().reset_index()
    recommendations = pd.merge(user_interests, events_df[['event_id', 'event_name', 'location', 'ticket_price']], on='event_id')
    recommendations = recommendations.sort_values(by='weight', ascending=False).head(n)
    recommendations.rename(columns={'weight': 'interest_score'}, inplace=True)
    
    # Calculate precision and recall if user_id is provided
    if user_id in user_relevant_items:
        relevant_items = user_relevant_items[user_id]
        precision_recall = calculate_precision_recall(recommendations['event_id'], relevant_items)
        print(f"Profile-Based - Precision: {precision_recall['precision']:.2f}, Recall: {precision_recall['recall']:.2f}")
    return recommendations


# Generate friend-based relevant events
def recommend_friends_events(user_id, n=10):
    # Get friends of the user
    friends = friends_df[friends_df['user_id'] == user_id]['friend_id'].tolist()
    
    # Find events liked or attended by friends
    friends_events = user_behavior_df[(user_behavior_df['user_id'].isin(friends)) & 
                                      (user_behavior_df['interaction_type'].isin(relevant_interactions))]
    
    # Aggregate interaction values to recommend top events among friends' interactions
    friends_event_scores = friends_events.groupby('event_id')['interaction_value'].sum().reset_index()
    recommendations = pd.merge(friends_event_scores, events_df[['event_id', 'event_name', 'location', 'ticket_price']], 
                               on='event_id').sort_values(by='interaction_value', ascending=False).head(n)
    
    # Calculate precision and recall if user_id is provided
    if user_id in user_relevant_items:
        relevant_items = user_relevant_items[user_id]
        precision_recall = calculate_precision_recall(recommendations['event_id'], relevant_items)
        print(f"Friends-Based - Precision: {precision_recall['precision']:.2f}, Recall: {precision_recall['recall']:.2f}")
    
    return recommendations[['event_id', 'event_name', 'location', 'ticket_price']]

# Example usage for friend-based recommendations
print("\nFriends-Based Recommendations for user 8:")
print(recommend_friends_events(user_id=6, n=10).to_string(index=False))


# Example usage for new and existing users
print("Popular Events Recommendations for new users:")
print(recommend_popular_events(n=10, user_id=6).to_string(index=False))

print("\nCategory-Based Recommendations for user 6:")
print(recommend_by_selected_category(user_id=6, n=10).to_string(index=False))

print("\nProfile-Based Recommendations for user 6:")
print(profile_based_recommend(user_id=6, n=10).to_string(index=False))



Friends-Based Recommendations for user 8:
Friends-Based - Precision: 0.30, Recall: 0.30
 event_id                                                  event_name       location  ticket_price
     14.0 Sip 'N Stroke |2pm - 5pm | Birmingham | Sip and Paint Party Birmingham, UK          35.0
     16.0                                    Mental Elf Coventry 2024   Coventry, UK          20.0
     25.0                                    Ford Yard Meet: The Hill  Liverpool, UK          20.0
     20.0                                              Feast of Judah     London, UK          20.0
     32.0                         Pigs in blankets eating competition  Liverpool, UK          40.0
     17.0                 Bollywood Time Machine – Coventry - Warwick    Warwick, UK          30.0
     19.0                                        The Big 5 Fundraiser Birmingham, UK          27.0
     47.0                   The Indians Are Coming – Coventry/Warwick    Bristol, UK          40.0
     12.0           

/var/folders/tf/nws1fbr52s33d747v5w6zhgh0000gp/T/ipykernel_22715/1410702473.py:52: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  recommendations = pd.merge(popular_events, events_df[['event_id', 'event_name', 'location', 'ticket_price']],
/var/folders/tf/nws1fbr52s33d747v5w6zhgh0000gp/T/ipykernel_22715/1410702473.py:80: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  recommendations = pd.merge(user_interests, events_df[['event_id', 'event_name', 'location', 'ticket_price']], on='event_id')
